## one-hot encode
- location (3.105) - cities and countries remove
    - countries = 90 (346 is NA) keep
- industry (groups = 131) boolean mask (group all with less than 30 into one group) --> create category with missings
- function (groups = 37)  create category with missings
- employment_type (groups = 5) 
- required_experience (groups = 7)
- required_education (groups = 13)

In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv("../raw_data/fake_job_postings.csv")
df.head(2)

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0


In [25]:
df['country'] = df['location'].astype(str).str[:2]
df['country'] = df['country'].replace("na", "Unknown")

In [26]:
# catagoreical columsn for One-Hot Encoding
categorical_columns = [
    'country',
    'industry',
    'function',
    'employment_type',
    'required_experience',
    'required_education'
]

In [15]:
# count how many times each category appears
df['industry'].nunique()
industry_counts = df['industry'].value_counts()
industry_counts

industry
Information Technology and Services    1734
Computer Software                      1376
Internet                               1062
Marketing and Advertising               828
Education Management                    822
                                       ... 
Sporting Goods                            1
Museums and Institutions                  1
Shipbuilding                              1
Alternative Dispute Resolution            1
Ranching                                  1
Name: count, Length: 131, dtype: int64

In [ ]:
# categrory appears less than 30 times
rare_industries = industry_counts[industry_counts < 30].index
# rare_industries
len(rare_industries)

67

In [17]:
# categrory appears 30 or more times
remaining_industries = industry_counts[industry_counts >= 30].index
# remaining_industries
len(remaining_industries)

64

In [ ]:
# Function to analyze categorical columns with unique value counts
def analyze_categories(df, column, threshold=30):
    print(f"--- Analyzing column: {column} ---")

    counts = df[column].value_counts()

    total_unique = df[column].nunique()
    rare = counts[counts < threshold].index
    remaining = counts[counts >= threshold].index

    print(f"Total unique categories: {total_unique}")
    print(f"Rare (<{threshold}): {len(rare)} categories")
    print(f"Remaining (≥{threshold}): {len(remaining)} categories")

    return counts, rare, remaining


In [21]:
industry_counts, rare_ind, remaining_ind = analyze_categories(df, 'industry')
function_counts, rare_func, remaining_func = analyze_categories(df, 'function')
education_counts, rare_edu, remaining_edu = analyze_categories(df, 'required_education')
experience_counts, rare_exp, remaining_exp = analyze_categories(df, 'required_experience')
employment_counts, rare_emp, remaining_emp = analyze_categories(df, 'employment_type')

--- Analyzing column: industry ---
Total unique categories: 131
Rare (<30): 67 categories
Remaining (≥30): 64 categories
--- Analyzing column: function ---
Total unique categories: 37
Rare (<30): 3 categories
Remaining (≥30): 34 categories
--- Analyzing column: required_education ---
Total unique categories: 13
Rare (<30): 4 categories
Remaining (≥30): 9 categories
--- Analyzing column: required_experience ---
Total unique categories: 7
Rare (<30): 0 categories
Remaining (≥30): 7 categories
--- Analyzing column: employment_type ---
Total unique categories: 5
Rare (<30): 0 categories
Remaining (≥30): 5 categories


In [ ]:
# filling missing values with "Unknown"
for col in categorical_columns:
    df[col] = df[col].fillna("Unknown")


In [30]:
# Function to group rare categories into "Other"
def group_rare_categories(df, column, threshold=30):
    counts = df[column].value_counts()
    rare_cats = counts[counts < threshold].index
    df[column] = df[column].replace(rare_cats, "Other")
    return df


In [ ]:
# Group rare categories for all categorical columns
for col in categorical_columns:
    df = group_rare_categories(df, col, threshold=30)

In [32]:
# One-hot encode
df_ohe = pd.get_dummies(df[categorical_columns], drop_first=True)

In [33]:
df_ohe.head()
df_ohe.shape
df_ohe.columns[:20]

Index(['country_AU', 'country_BE', 'country_BR', 'country_CA', 'country_DE',
       'country_DK', 'country_EE', 'country_EG', 'country_ES', 'country_FR',
       'country_GB', 'country_GR', 'country_HK', 'country_IE', 'country_IL',
       'country_IN', 'country_IT', 'country_NL', 'country_NZ',
       'country_Other'],
      dtype='object')